# Merge Forum Data
This notebook merges the data from individual netmums databases into netmumsTables.db

## Changes
- 2021-02-08: Created
- 2021-02-09: Added merge loop
- 2021-02-26: Reformated loop to new data
## TODO
- 

## Imports

In [1]:
import sqlite3
from pathlib import Path
from scraping import create_connection
from netmums import set_up_merged_db
import pandas as pd

## File Locations

In [2]:
p = Path.cwd()
path_parent = p.parents[0]

## Clean Data

Remove duplicate posts and renumber post counts

In [3]:
has_duplicate_posts_sql = """
    SELECT thread_id, post_id, user_url, date_created, COUNT(*)
    FROM posts
    GROUP BY thread_id, post_id, user_url, date_created
    HAVING COUNT(*)>1;
"""

In [4]:
drop_duplicate_posts_sql = """
    DELETE FROM posts
    WHERE id NOT IN (
        SELECT min(id)
        FROM posts
        GROUP BY thread_id, post_id, user_url, date_created
    );
"""

In [5]:
select_post_threads_sql = """
    SELECT id, post_count
    FROM posts
    WHERE thread_id={}
"""

In [6]:
update_post_count_sql = """
    UPDATE posts
    SET post_count={0}
    WHERE id={1} 
"""

In [7]:
for i in range(1, 6):
    db = "netmums0{}.db".format(i)
    path_db = str(path_parent / "database" / db)
    conn = create_connection(path_db)
    cur = conn.cursor()
    has_duplicate_posts = pd.read_sql(has_duplicate_posts_sql, conn)
    if not has_duplicate_posts.empty:
        unique_threads = list(has_duplicate_posts.thread_id.unique())
        cur.execute(drop_duplicate_posts_sql)
        for thread_id in unique_threads:
            id_df = pd.read_sql(select_post_threads_sql.format(thread_id), conn)
            id_df['post_count'] = id_df['post_count'].astype(int)
            id_df.sort_values(by=['post_count'], inplace=True)
            for i, r in id_df.iterrows():
                cur.execute(update_post_count_sql.format(i + 1, r['id']))
        conn.commit()
    conn.close()

Remove duplicate users, quotes, and links

In [8]:
duplicates_sql = '''
    DELETE FROM users
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM users
        GROUP BY name, user_url
    );
    DELETE FROM quotes
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM quotes
        GROUP BY thread_id, quoting_id, quoted_id, quoted_user, quoted_text, citation_n
    );
    DELETE FROM links
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM links
        GROUP BY thread_id, post_id, link_count, link_text, link_url
    );
'''

In [9]:
for i in range(1, 6):
    db = "netmums0{}.db".format(i)
    path_db = str(path_parent / "database" / db)
    conn = create_connection(path_db)
    cur = conn.cursor()
    cur.executescript(duplicates_sql)
    conn.commit()
    conn.close()

Match quotes to post_id

1. select quotes with no quoted_id

Problem: post_ids are not unique :(((((((
Solution: match links and quotes to correct post
- quotes by text
- links by order

In [10]:
quotes_sql = '''
    SELECT *
    FROM quotes
    WHERE quoted_id=""
'''

In [11]:
db = "netmums01 (copy).db"
path_db = str(path_parent / "database" / db)
conn = create_connection(path_db)
quotes = pd.read_sql(quotes_sql, conn)
# no_quoted['dup'] = no_quoted.post_id_old.duplicated(keep=False)

In [ ]:
quotes.head()

In [28]:
get_posts = '''
    SELECT
        p.thread_id AS thread_id,
        p.post_count AS post_count,
        p.body AS body
    FROM posts AS p
    LEFT JOIN users AS u
        ON p.user_url = u.user_url
    WHERE
        p.thread_id={0}
        AND u.name="{1}"
        AND p.post_count<{2};
'''

In [38]:
get_posts_anon = '''
    SELECT
        thread_id,
        post_count,
        body
    FROM posts
    WHERE
        thread_id={0}
        AND user_url="Anonymous"
        AND post_count<{1};
'''

In [26]:
quotes.iloc[3]["post_count"]

186

In [41]:
import re
for i in range(100):
    thread_id = quotes.iloc[i]["thread_id"]
    name = quotes.iloc[i]["quoted_user"]
    post_count = quotes.iloc[i]["post_count"]
    quoted_text = quotes.iloc[i]["quoted_text"]
    if name == "Anonymous":
        posts = pd.read_sql(get_posts_anon.format(thread_id, post_count), conn)
    else:
        posts = pd.read_sql(get_posts.format(thread_id, name, post_count), conn)
    if len(posts.index) == 1:
        # yay, found correct post
        pass
    else:
        # match text
        print("row:", i)
        matches = posts['body'].str.contains(quoted_text, flags=re.IGNORECASE, regex=False)
        print(matches.value_counts())

row: 20
False    72
True      1
Name: body, dtype: int64
row: 21
False    99
True      1
Name: body, dtype: int64
row: 22
False    14
True      1
Name: body, dtype: int64
row: 23
False    21
True      1
Name: body, dtype: int64
row: 24
False    56
True      1
Name: body, dtype: int64
row: 27
False    59
True      1
Name: body, dtype: int64
row: 28
True     1
False    1
Name: body, dtype: int64
row: 29
False    63
True      1
Name: body, dtype: int64
row: 30
False    2
True     1
Name: body, dtype: int64
row: 32
False    75
True      1
Name: body, dtype: int64
row: 33
False    76
True      1
Name: body, dtype: int64
row: 34
False    80
True      1
Name: body, dtype: int64


In [ ]:
# no_quoted_sql = """
#     SELECT quoting_id, quoted_user, quoted_text
#     FROM quotes
#     WHERE quoted_id=""
# """

In [ ]:
no_quoted_sql = """
    SELECT
        p.thread_id as thread_id,
        p.id as post_id,
        p.post_id as post_id_old,
        q.quoted_user as quoted_user,
        q.quoted_text as quoted_text
    FROM quotes as q
    LEFT JOIN posts as p
    ON q.quoting_id = p.post_id
    WHERE q.quoted_id=""
"""

In [ ]:
count = 0
grouped = no_quoted.groupby('post_id_old')
for n, g in grouped:
    if count > 10:
        break
    if g.shape[0] > 1:
        possible_threads = g.thread_id.unique()
        if len(threads) > 1:
            count += 1
            print()
            print(g)

In [ ]:
no_quoted.dup.value_counts()

In [ ]:
count = 0
for i, r in no_quoted.iterrows():
    if count > 10:
        break
    if r["dup"]:
        count += 1
        potential_threads = no_quoted.loc())

In [ ]:
links_sql = """
    SELECT
        post_id,
        link_url
    FROM links
    LEFT JOIN posts as p
    ON l.post_id = p.post_id
    WHERE p.body LIKE '%::link_%';
"""

In [ ]:
links_sql = """
    SELECT
        id,
        post_id,
        link_count,
        link_url
    FROM links;
"""

In [ ]:
posts_with_links_sql = """
    SELECT
        id,
        thread_id,
        post_id,
        body
    FROM posts
    WHERE body LIKE '%::link_%';
"""

In [ ]:
db = "netmums01 (copy).db".format(i)
path_db = str(path_parent / "database" / db)
conn = create_connection(path_db)
links = pd.read_sql(links_sql, conn)
posts = pd.read_sql(posts_with_links_sql, conn)

In [ ]:
posts["link_counts"] = posts.body.str.count("::link_[0-9]*::")

In [ ]:
posts["link_counts"].value_counts()

In [ ]:
unique_ids = list(posts.post_id.unique())

In [ ]:
for uid in unique_ids:
    size_links = 

## Merge Databases

In [ ]:
db = "netmums-merged.db"
path_db = str(path_parent / "database" / db)
conn = create_connection(path_db)
cur = conn.cursor()
loop_sql = '''
    ATTACH DATABASE "{0}" AS db{1};
    INSERT INTO main.users (name, data_user_id, url) SELECT name, data_user_id, url FROM db{1}.users;
    INSERT INTO main.quotes (quoting_id, quoted_id) SELECT quoting_id, quoted_id FROM db{1}.quotes;
    INSERT INTO main.posts (thread_id, post_id, post_count, data_user_id, date_created, date_recorded, body) SELECT thread_id, post_id, post_count, data_user_id, date_created, date_recorded, body FROM db{1}.posts;
    DETACH DATABASE db{1};
'''

In [ ]:
duplicates_sql = '''
    DELETE FROM users
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM users
        GROUP BY name, data_user_id, url
    );
    DELETE FROM quotes
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM quotes
        GROUP BY quoting_id, quoted_id
    );
'''

In [ ]:
for i in range(1, 11):
    n = pad(i)
    fn = "netmumsTables-{}.db".format(n)
    path_add = str(path_parent / "database" / fn)
    cur.executescript(loop_sql.format(path_add, n))

In [ ]:
cur.executescript(duplicates_sql)

In [ ]:
conn.commit()

In [ ]:
conn.close()